In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 waste_cnt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)  
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50130250

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,287127.8086
2018-02-28,264529.9658
2018-03-31,309677.5996
2018-04-30,326690.3964
2018-05-31,298018.0474
2018-06-30,247779.7541
2018-07-31,194576.6543
2018-08-31,440089.6185
2018-09-30,410880.1503


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31    287127.8086
 2018-02-28    264529.9658
 2018-03-31    309677.5996
 2018-04-30    326690.3964
 2018-05-31    298018.0474
 2018-06-30    247779.7541
 2018-07-31    194576.6543
 2018-08-31    440089.6185
 2018-09-30    410880.1503
 2018-10-31    353250.6947
 2018-11-30    339220.1524
 2018-12-31    293304.5548
 2019-01-31    456492.0846
 2019-02-28    383855.6543
 2019-03-31    470441.5424
 2019-04-30    435325.6361
 2019-05-31    491081.4313
 2019-06-30    369632.6879
 2019-07-31    282652.1468
 2019-08-31    591568.7562
 2019-09-30    464691.4471
 2019-10-31    473322.3577
 2019-11-30    461246.2650
 2019-12-31    383292.9726
 2020-01-31    454054.2490
 2020-02-29    445784.6368
 2020-03-31    452000.6555
 2020-04-30    492968.1492
 2020-05-31    438869.7804
 2020-06-30    362377.4286
 2020-07-31    374882.7590
 2020-08-31    432066.9450
 2020-09-30    439286.1887
 2020-10-31    449787.0901
 2020-11-30    385702.6556
 2020-12-31    445228.8540

# ARIMA

In [6]:
# ARIMA에 적합한 log로 바꾸는 과정
sub_df_log = np.log1p(sub_df[:])
#sub_df_float.tail()

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df_log)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -3.049185
p-value : 0.030549
Critical Values :
	1%: -3.653520
	5%: -2.957219
	10%: -2.617588


In [8]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df_log,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df_log)

 ARIMA(0,1,0)(0,0,0)[1] intercept   : AIC=1.562, Time=0.05 sec
 ARIMA(0,1,1)(0,0,0)[1] intercept   : AIC=-14.500, Time=0.14 sec
 ARIMA(0,1,2)(0,0,0)[1] intercept   : AIC=-12.629, Time=0.16 sec
 ARIMA(0,1,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.35 sec
 ARIMA(0,1,4)(0,0,0)[1] intercept   : AIC=inf, Time=0.41 sec
 ARIMA(0,1,5)(0,0,0)[1] intercept   : AIC=inf, Time=0.44 sec
 ARIMA(1,1,0)(0,0,0)[1] intercept   : AIC=-2.748, Time=0.05 sec
 ARIMA(1,1,1)(0,0,0)[1] intercept   : AIC=-11.424, Time=0.18 sec
 ARIMA(1,1,2)(0,0,0)[1] intercept   : AIC=inf, Time=0.33 sec
 ARIMA(1,1,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.36 sec
 ARIMA(1,1,4)(0,0,0)[1] intercept   : AIC=inf, Time=0.43 sec
 ARIMA(2,1,0)(0,0,0)[1] intercept   : AIC=-5.141, Time=0.09 sec
 ARIMA(2,1,1)(0,0,0)[1] intercept   : AIC=inf, Time=0.25 sec
 ARIMA(2,1,2)(0,0,0)[1] intercept   : AIC=inf, Time=0.36 sec
 ARIMA(2,1,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.38 sec
 ARIMA(3,1,0)(0,0,0)[1] intercept   : AIC=-7.203, Time=0.09 sec
 

ARIMA(order=(4, 1, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [9]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df_log.long_term_frgn.values, order=(4,1,0))    

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                             ARIMA Model Results                              
Dep. Variable:                    D.y   No. Observations:                   41
Model:                 ARIMA(4, 1, 0)   Log Likelihood                  13.293
Method:                       css-mle   S.D. of innovations              0.172
Date:                Sat, 11 Sep 2021   AIC                            -14.586
Time:                        04:24:38   BIC                             -4.305
Sample:                             1   HQIC                           -10.842
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0093      0.008      1.154      0.248      -0.007       0.025
ar.L1.D.y     -0.7436      0.138     -5.405      0.000      -1.013      -0.474
ar.L2.D.y     -0.6955      0.155     -4.473      0.0

# 예측하기

In [10]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([12.90857416, 13.01399457]), array([0.17168884, 0.17724098]), array([[12.57207022, 13.24507809],
       [12.66660863, 13.36138051]]))


In [11]:
print( np.exp(12.90857416) - 1 )   
print( np.exp(13.01399457) - 1 )  

403758.2889735555
448647.3028830983
